In [57]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset

from tqdm import tqdm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
def torch_seed(seed=123, deter = False):

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = deter
    torch.use_deterministic_algorithms = deter

In [3]:
import pandas as pd
from urllib import request

In [4]:
input_dim = 5
hidden_size = 3

inputs = torch.rand(1, 3, 5)
print(inputs)
print(inputs.dtype)

tensor([[[0.5496, 0.0869, 0.9621, 0.6901, 0.2306],
         [0.8452, 0.2220, 0.4614, 0.8164, 0.3041],
         [0.9017, 0.4805, 0.3105, 0.2364, 0.2888]]])
torch.float32


In [5]:
input_dim = 5
hidden_size = 3
lstm = nn.LSTM(input_dim, hidden_size, batch_first=True)

In [6]:
lstm = nn.LSTM(input_dim, hidden_size, batch_first=True)
print(list(lstm.parameters()))

[Parameter containing:
tensor([[-0.0023, -0.1343, -0.1855,  0.4428, -0.4627],
        [-0.1614, -0.2065,  0.0769,  0.4788,  0.1603],
        [-0.5254,  0.5322, -0.4646, -0.5736, -0.1825],
        [ 0.1046,  0.0274, -0.0320, -0.2019, -0.1655],
        [ 0.0630, -0.4846,  0.1456,  0.3715,  0.0569],
        [-0.2977,  0.0971, -0.3037,  0.0282, -0.1060],
        [ 0.4831,  0.0782,  0.4329, -0.1310, -0.2854],
        [ 0.1873, -0.4835,  0.0755,  0.2951,  0.0260],
        [-0.5477, -0.5100, -0.2534, -0.0786,  0.3035],
        [-0.3348,  0.5102, -0.4349,  0.5476,  0.1434],
        [ 0.3470,  0.3339, -0.4148,  0.5323,  0.0888],
        [-0.3449,  0.5043,  0.1783, -0.3644,  0.0795]], requires_grad=True), Parameter containing:
tensor([[-0.1692, -0.1737,  0.2689],
        [ 0.2140,  0.4775, -0.1735],
        [ 0.5588, -0.3172,  0.4728],
        [ 0.0223,  0.3172, -0.0376],
        [ 0.1760,  0.3110, -0.1314],
        [-0.3359, -0.3798,  0.2941],
        [-0.0270, -0.4825, -0.1552],
        [-0.36

In [7]:
request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt",
                    filename="ratings_train.txt")
request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt",
                    filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7e2ffe41c8d0>)

In [8]:
train_data = pd.read_table('ratings_train.txt', sep = "\t", nrows = 10000)
test_data = pd.read_table('ratings_test.txt', sep = "\t", nrows = 10000)
print('총 샘플의 수 :',len(train_data))
train_data.head()

총 샘플의 수 : 10000


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [9]:
train_data.dropna(inplace=True, how = "any")
train_data.drop_duplicates(subset=['document'], inplace=True)
print('총 샘플의 수 :',len(train_data))
train_data.head()

총 샘플의 수 : 9918


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [10]:
### 한글과 공백을 제외하고 모두 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"," ",
                                                             regex=True)
train_data.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [11]:
train_data['document'] = train_data['document'].str.strip()
train_data['document'] = train_data['document'].replace('', np.nan)
train_data.dropna(how = 'any', inplace=True)
print(train_data.isnull().sum())
print(train_data.shape) # (145393, 3)

id          0
document    0
label       0
dtype: int64
(9858, 3)


In [12]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 43.0 MB/s eta 0:00:00


In [13]:
from konlpy.tag import Okt
from tqdm import tqdm

In [14]:
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']
okt = Okt()
okt.morphs("교도소 이야기구먼 솔직히 재미는 없다평점 조정")

['교도소', '이야기', '구먼', '솔직히', '재미', '는', '없다', '평점', '조정']

In [15]:
X_data = []
for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_data.append(stopwords_removed_sentence)

100%|██████████| 9858/9858 [00:25<00:00, 392.66it/s]


In [16]:
y_data = train_data['label']

print(len(X_data))
print(len(y_data))

print(X_data[:5])
print(y_data[:5])

9858
9858
[['아', '더빙', '진짜', '짜증나다', '목소리'], ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다'], ['너', '무재', '밓었', '다그', '래서', '보다', '추천'], ['교도소', '이야기', '구먼', '솔직하다', '재미', '없다', '평점', '조정'], ['사이', '몬페', '그', '익살스럽다', '연기', '돋보이다', '영화', '스파이더맨', '에서', '늙다', '보이다', '하다', '커스틴', '던스트', '너무나도', '이쁘다', '보이다']]
0    0
1    1
2    0
3    0
4    1
Name: label, dtype: int64


In [17]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_data, y_data, test_size=0.2,
                                                      random_state=0, stratify=y_data)
print("X_train shape = ", len(X_train))
print("X_valid shape = ", len(X_valid))
print("y_train shape = ", len(y_train))
print("y_valid shape = ", len(y_valid))

X_train shape =  7886
X_valid shape =  1972
y_train shape =  7886
y_valid shape =  1972


In [18]:
from collections import Counter

word_list = []
for sent in X_train:
    for word in sent:
      word_list.append(word)

word_counts = Counter(word_list) # == len(set(word_list))
print('총 단어수 :', len(word_counts))
display(word_counts)

총 단어수 : 10876


Counter({'감동': 258,
         '교육': 10,
         '계': 15,
         '비판': 13,
         '사랑': 158,
         '손대다': 3,
         '방향': 6,
         '모두': 51,
         '에서': 362,
         '확실하다': 24,
         '거': 229,
         '없이': 81,
         '어정쩡하다': 2,
         '영화': 2678,
         '바라보다': 9,
         '만': 497,
         '하다': 2241,
         '라면': 35,
         '대체': 27,
         '극영화': 1,
         '로': 406,
         '왜': 271,
         '만들다': 335,
         '거야': 5,
         '재미있다': 192,
         '허풍': 2,
         '떨다': 7,
         '말다': 57,
         '이런': 94,
         '류': 38,
         '솔직하다': 73,
         '서방': 1,
         '애': 69,
         '어떻다': 85,
         '볼': 157,
         '쪽팔리다': 7,
         '한국인': 5,
         '진정하다': 20,
         '개신교': 3,
         '아니다': 420,
         '새롭다': 29,
         '종교': 12,
         '숭배': 1,
         '수컷': 1,
         '짝짓기': 2,
         '경쟁': 2,
         '부르다': 14,
         '파국': 1,
         '랄': 9,
         '끄다': 74,
         '나르다': 13,
         '복제': 3,

In [19]:
print('훈련 데이터에서의 단어 영화의 등장 횟수 :', word_counts['영화']) # class Counter (= dictionary)
print('훈련 데이터에서의 단어 공감의 등장 횟수 :', word_counts['공감'])

훈련 데이터에서의 단어 영화의 등장 횟수 : 2678
훈련 데이터에서의 단어 공감의 등장 횟수 : 63


In [20]:
vocab = sorted(word_counts, key=word_counts.get, reverse=True)
vocab

['영화',
 '보다',
 '하다',
 '없다',
 '이다',
 '있다',
 '좋다',
 '정말',
 '너무',
 '만',
 '재밌다',
 '같다',
 '점',
 '되다',
 '적',
 '진짜',
 '아니다',
 '으로',
 '로',
 '않다',
 '나오다',
 '연기',
 '에서',
 '만들다',
 '평점',
 '나',
 '최고',
 '것',
 '내',
 '안',
 '그',
 '못',
 '사람',
 '스토리',
 '드라마',
 '왜',
 '보고',
 '감동',
 '생각',
 '감독',
 '말',
 '이렇다',
 '때',
 'ㅋㅋ',
 '그냥',
 '아깝다',
 '아',
 '거',
 '재미없다',
 '시간',
 '배우',
 '더',
 '내용',
 '중',
 '재미',
 '요',
 '자다',
 '지루하다',
 '가다',
 '까지',
 '재미있다',
 '하고',
 '뭐',
 '들다',
 '모르다',
 '주다',
 '작품',
 '쓰레기',
 '수',
 '좀',
 '알다',
 '사랑',
 '하나',
 '볼',
 '싶다',
 '이건',
 'ㅋ',
 '잘',
 '마지막',
 '정도',
 '그렇다',
 '개',
 '차다',
 '액션',
 '연출',
 '돈',
 '이렇게',
 '저',
 '다시',
 '걸',
 '주인공',
 '최악',
 '안되다',
 '지금',
 '완전',
 '기',
 '많다',
 '나다',
 '받다',
 '느낌',
 '오다',
 '처음',
 '장면',
 'ㅠㅠ',
 '역시',
 'ㅋㅋㅋ',
 '별',
 '명작',
 '이야기',
 '인데',
 '라',
 '넘다',
 '별로',
 '부터',
 '일',
 'ㅡㅡ',
 '면',
 '먹다',
 '남다',
 '이나',
 '좋아하다',
 '꼭',
 '괜찮다',
 '년',
 '버리다',
 '또',
 '아름답다',
 '인생',
 '이해',
 '끝',
 '난',
 '느끼다',
 '라고',
 '이영화',
 '이런',
 '무슨',
 '그리고',
 '멋지다',
 '해주다',
 '야',
 '서',
 '전',
 '줄',
 '많이',


In [21]:
###
word_to_index = {}
word_to_index['<PAD>'] = 0
word_to_index['<UNK>'] = 1

for index, word in enumerate(vocab) :
  word_to_index[word] = index + 2

vocab_size = len(word_to_index)
print('패딩 토큰과 UNK 토큰을 고려한 단어 집합의 크기 :', vocab_size)

패딩 토큰과 UNK 토큰을 고려한 단어 집합의 크기 : 10878


In [22]:
print(word_to_index)
print('단어 <PAD>와 맵핑되는 정수 :', word_to_index['<PAD>'])
print('단어 <UNK>와 맵핑되는 정수 :', word_to_index['<UNK>'])
print('단어 영화와 맵핑되는 정수 :', word_to_index['영화'])

{'<PAD>': 0, '<UNK>': 1, '영화': 2, '보다': 3, '하다': 4, '없다': 5, '이다': 6, '있다': 7, '좋다': 8, '정말': 9, '너무': 10, '만': 11, '재밌다': 12, '같다': 13, '점': 14, '되다': 15, '적': 16, '진짜': 17, '아니다': 18, '으로': 19, '로': 20, '않다': 21, '나오다': 22, '연기': 23, '에서': 24, '만들다': 25, '평점': 26, '나': 27, '최고': 28, '것': 29, '내': 30, '안': 31, '그': 32, '못': 33, '사람': 34, '스토리': 35, '드라마': 36, '왜': 37, '보고': 38, '감동': 39, '생각': 40, '감독': 41, '말': 42, '이렇다': 43, '때': 44, 'ㅋㅋ': 45, '그냥': 46, '아깝다': 47, '아': 48, '거': 49, '재미없다': 50, '시간': 51, '배우': 52, '더': 53, '내용': 54, '중': 55, '재미': 56, '요': 57, '자다': 58, '지루하다': 59, '가다': 60, '까지': 61, '재미있다': 62, '하고': 63, '뭐': 64, '들다': 65, '모르다': 66, '주다': 67, '작품': 68, '쓰레기': 69, '수': 70, '좀': 71, '알다': 72, '사랑': 73, '하나': 74, '볼': 75, '싶다': 76, '이건': 77, 'ㅋ': 78, '잘': 79, '마지막': 80, '정도': 81, '그렇다': 82, '개': 83, '차다': 84, '액션': 85, '연출': 86, '돈': 87, '이렇게': 88, '저': 89, '다시': 90, '걸': 91, '주인공': 92, '최악': 93, '안되다': 94, '지금': 95, '완전': 96, '기': 97, '많다': 98, '나다': 99, '받다': 100, 

In [23]:
def texts_to_sequences(tokenized_X_data, word_to_index):
  encoded_X_data = []
  for sent in tokenized_X_data:
    index_sequences = []
    for word in sent:
      try:
          index_sequences.append(word_to_index[word])
      except KeyError:
          index_sequences.append(word_to_index['<UNK>'])
    encoded_X_data.append(index_sequences)
  return encoded_X_data

In [24]:
encoded_X_train = texts_to_sequences(X_train, word_to_index)
encoded_X_valid = texts_to_sequences(X_valid, word_to_index)

In [31]:
index_to_word = {}
for key, value in word_to_index.items():
  index_to_word[value] = key
index_to_word

{0: '<PAD>',
 1: '<UNK>',
 2: '영화',
 3: '보다',
 4: '하다',
 5: '없다',
 6: '이다',
 7: '있다',
 8: '좋다',
 9: '정말',
 10: '너무',
 11: '만',
 12: '재밌다',
 13: '같다',
 14: '점',
 15: '되다',
 16: '적',
 17: '진짜',
 18: '아니다',
 19: '으로',
 20: '로',
 21: '않다',
 22: '나오다',
 23: '연기',
 24: '에서',
 25: '만들다',
 26: '평점',
 27: '나',
 28: '최고',
 29: '것',
 30: '내',
 31: '안',
 32: '그',
 33: '못',
 34: '사람',
 35: '스토리',
 36: '드라마',
 37: '왜',
 38: '보고',
 39: '감동',
 40: '생각',
 41: '감독',
 42: '말',
 43: '이렇다',
 44: '때',
 45: 'ㅋㅋ',
 46: '그냥',
 47: '아깝다',
 48: '아',
 49: '거',
 50: '재미없다',
 51: '시간',
 52: '배우',
 53: '더',
 54: '내용',
 55: '중',
 56: '재미',
 57: '요',
 58: '자다',
 59: '지루하다',
 60: '가다',
 61: '까지',
 62: '재미있다',
 63: '하고',
 64: '뭐',
 65: '들다',
 66: '모르다',
 67: '주다',
 68: '작품',
 69: '쓰레기',
 70: '수',
 71: '좀',
 72: '알다',
 73: '사랑',
 74: '하나',
 75: '볼',
 76: '싶다',
 77: '이건',
 78: 'ㅋ',
 79: '잘',
 80: '마지막',
 81: '정도',
 82: '그렇다',
 83: '개',
 84: '차다',
 85: '액션',
 86: '연출',
 87: '돈',
 88: '이렇게',
 89: '저',
 90: '다시',
 91: '걸',
 

In [33]:
decoded_sample = [index_to_word[word] for word in encoded_X_train[0]]

print(X_train[0])
print(decoded_sample)

['감동', '교육', '계', '비판', '사랑', '손대다', '방향', '모두', '에서', '확실하다', '거', '없이', '어정쩡하다', '영화', '바라보다', '만', '하다', '거', '라면', '대체', '극영화', '로', '왜', '만들다', '거야']
['감동', '교육', '계', '비판', '사랑', '손대다', '방향', '모두', '에서', '확실하다', '거', '없이', '어정쩡하다', '영화', '바라보다', '만', '하다', '거', '라면', '대체', '극영화', '로', '왜', '만들다', '거야']


In [35]:
## zero padding

encoded_X_train[:3]

[[39,
  1186,
  843,
  976,
  73,
  2771,
  1718,
  272,
  24,
  573,
  49,
  164,
  3651,
  2,
  1279,
  11,
  4,
  49,
  399,
  514,
  5321,
  20,
  37,
  25,
  1949],
 [62, 3652, 1537, 247],
 [136,
  373,
  2,
  192,
  5322,
  202,
  155,
  75,
  1538,
  1950,
  666,
  2772,
  18,
  479,
  1045,
  25,
  5323,
  4]]

In [36]:
max_len = 30
def pad_sequences(sentences, max_len):
  features = np.zeros((len(sentences), max_len), dtype=np.int32)
  for index, sentence in enumerate(sentences):
    if len(sentence) != 0:
      features[index, :len(sentence)] = np.array(sentence)[:max_len]
  return features

padded_X_train = pad_sequences(encoded_X_train, max_len)
padded_X_valid = pad_sequences(encoded_X_valid, max_len)

print(padded_X_train.shape)
print(padded_X_train.shape)

(7886, 30)
(7886, 30)


In [40]:
train_label_tensor = torch.tensor(np.array(y_train))
print(train_label_tensor.dtype)

valid_label_tensor = torch.tensor(np.array(y_valid))
print(valid_label_tensor.dtype)

torch.int64
torch.int64


In [41]:
embedding_dim = 100
hidden_dim = 128
output_dim = 2
lr = 0.1
num_epochs = 10

In [42]:
class TextClassifier(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, x):
    x = self.embedding(x)
    lstm_out, (hidden, cell) = self.lstm(x)

    last_hidden = hidden.squeeze(0)
    x = self.fc(last_hidden)

    return x

In [43]:
padded_X_train

array([[  39, 1186,  843, ...,    0,    0,    0],
       [  62, 3652, 1537, ...,    0,    0,    0],
       [ 136,  373,    2, ...,    0,    0,    0],
       ...,
       [ 221,   60,  459, ...,    0,    0,    0],
       [ 365,  910,  386, ...,    0,    0,    0],
       [ 312,  361,  235, ...,    0,    0,    0]], dtype=int32)

In [46]:
encoded_train = torch.tensor(padded_X_train)
encoded_valid = torch.tensor(padded_X_valid)

train_label_tensor = torch.tensor(np.array(y_train))
valid_label_tensor = torch.tensor(np.array(y_valid))

In [48]:
## Datasets

train_dataset = TensorDataset(encoded_train, train_label_tensor)
valid_dataset = TensorDataset(encoded_valid, valid_label_tensor)

In [53]:
## DataLoader

batch_size = 8

train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                               shuffle=True,
                                               batch_size=batch_size)

valid_dataloader = torch.utils.data.DataLoader(valid_dataset,
                                               shuffle=False,
                                               batch_size=batch_size)

In [54]:
vocab_size = len(word_to_index)
print(vocab_size)

10878


In [55]:
model = TextClassifier(vocab_size, embedding_dim, hidden_dim, output_dim).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [58]:
def calculate_accuracy(outputs, labels):
  predicted = outputs.argmax(-1)
  acc = (predicted == labels).float().mean().item()
  return acc

In [71]:
num_epochs = 20

best_val_loss = float('inf')

history = np.zeros((0, 5))

for epoch in range(num_epochs):
  train_loss = 0
  train_acc = 0
  val_loss = 0
  val_acc = 0
  train_total = 0
  val_total = 0

  for batch_X, batch_y in tqdm(train_dataloader):
    batch_X, batch_y = batch_X.to(device), batch_y.to(device)

    outputs = model(batch_X)
    loss = criterion(outputs, batch_y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    train_loss += loss.item()
    train_acc += calculate_accuracy(outputs, batch_y)
    train_total += batch_y.size(0)

  train_loss = train_loss / len(train_dataloader)
  train_acc = train_acc / train_total

  with torch.no_grad():
    for batch_X, batch_y in valid_dataset:
      batch_X, batch_y = batch_X.to(device), batch_y.to(device)

      outputs = model(batch_X)
      loss = criterion(outputs, batch_y)

      val_loss += loss.item()
      val_acc += calculate_accuracy(outputs, batch_y)
      val_total += 1

    val_loss = val_loss / len(valid_dataloader)
    val_acc = val_acc / val_total

  print(f"Epoch = {epoch}, train loss = {train_loss:.3f}, train acc = {train_acc:.3f}")
  print(f"Epoch = {epoch}, val loss = {val_loss:.3f}, val acc = {val_acc:.3f}")
  item = np.array([epoch, train_loss, train_acc, val_loss, val_acc])
  history = np.vstack((history, item))

  if val_loss < best_val_loss:
    best_val_loss = val_loss
    print(f"{epoch}: model saved!")
    torch.save(model.state_dict(), 'best_model_checkpoint.pt')

100%|██████████| 986/986 [00:02<00:00, 423.58it/s]


Epoch = 0, train loss = 0.001, train acc = 0.125
Epoch = 0, val loss = 15.793, val acc = 0.767
0: model saved!


100%|██████████| 986/986 [00:02<00:00, 432.71it/s]


Epoch = 1, train loss = 0.001, train acc = 0.125
Epoch = 1, val loss = 16.216, val acc = 0.766


100%|██████████| 986/986 [00:02<00:00, 432.77it/s]


...

Epoch = 17, train loss = 0.001, train acc = 0.125
Epoch = 17, val loss = 15.831, val acc = 0.770


100%|██████████| 986/986 [00:02<00:00, 432.78it/s]


Epoch = 18, train loss = 0.001, train acc = 0.125
Epoch = 18, val loss = 16.049, val acc = 0.767


100%|██████████| 986/986 [00:02<00:00, 433.29it/s]


Epoch = 19, train loss = 0.001, train acc = 0.125
Epoch = 19, val loss = 16.303, val acc = 0.772
